<a href="https://colab.research.google.com/github/CienciaDatosUdea/002_EstudiantesAprendizajeEstadistico/blob/main/semestre2024-2/Sesiones/Sesion_16_Reduccion_Dimensionalidad_PCA_Perm_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt

# Reduccion de dimensionalidad

In [ ]:
# Ejemplo de data set sintetico con variables aleatorias y variables relacionadas
n = 100
# variable causal con ruido gausiano pequeño
x1 = np.linspace(-1, 1,n) + np.random.RandomState().normal(0, 0.05, n)
# variable causal con un ruido grande uniforme
x2 = np.linspace(-1, 1,n) + np.random.RandomState().uniform(-0.5, 0.5, n)
# variable pura aleatoria gausiana
x3 = np.random.RandomState().normal(0, 1, n)
#variable pura aleatoria uniforme
x4 =  np.random.RandomState().uniform(0, 1, n)
#target y=5x+ruido
y = np.array([0] * (n // 2) + [1] * (n // 2))
#y =5*np.linspace(-1, 1,n) + np.random.RandomState().normal(0, 0.1, n)

In [ ]:
#verifiquen como se ve la relacion entre cada x y el target y
#print(x4,y)
plt.figure()
plt.plot(x1, y,"bo")
plt.xlabel("variable")
plt.ylabel("target")
plt.show()

## Permutation Test para clasificadores
https://www.jmlr.org/papers/volume11/ojala10a/ojala10a.pdf

Este test nos permite encontrar variables que estan muy poco correlacionadas con la variable objetivo. Se basa en un test combinatorio por lo cual a pesar de ser un test riguroso es dificil de aplicar a datasets muy grandes.


In [ ]:
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
#reshape for the test
x1_perm = x1.reshape(-1,1)
x2_perm = x2.reshape(-1,1)
x3_perm = x3.reshape(-1,1)
x4_perm = x4.reshape(-1,1)

In [ ]:
def permutation_test(x,y):
    estimator = LogisticRegression()
    score, permutation_scores, pvalue = permutation_test_score(

    estimator, x, y, random_state=0
    )
    print(f"Original Score: {score:.3f}")
    print(
        f"Permutation Scores: {permutation_scores.mean():.3f} +/- "
        f"{permutation_scores.std():.3f}"
        )
    print(f"P-value: {pvalue:.3f}")
    return None

In [ ]:
print("X1")
permutation_test(x1_perm,y)

print("X2")
permutation_test(x2_perm,y)

print("X3")
permutation_test(x3_perm,y)

print("X4")
permutation_test(x4_perm,y)



The p-value represents the fraction of randomized data sets where the estimator performed as well or better than in the original data. A small p-value suggests that there is a real dependency between features and targets which has been used by the estimator to give good predictions. A large p-value may be due to lack of real dependency between features and targets or the estimator was not able to use the dependency to give good predictions.

Basados en el bajo valor p de x1 y x2 versus el alto valor p de x3 y x4 podemos reducir el numero de variables a x1 y x2

## Principal Component Analysis PCA

El análisis de componentes principales (PCA) es una técnica estadística que se utiliza para reducir la dimensionalidad de un conjunto de datos, es decir, para representar los datos con menos variables sin perder mucha información. El PCA busca las combinaciones lineales de las variables originales que capturan la mayor parte de la variación o la dispersión de los datos. Estas combinaciones se llaman componentes principales, y se ordenan de mayor a menor importancia. El PCA se puede usar para explorar la estructura de los datos, para visualizarlos en un espacio de baja dimensión, o para prepararlos para otros análisis como la regresión o la clasificación.

Para entender mejor el PCA, puedes consultar la sección 3.2.1 y el apendice A de https://arxiv.org/pdf/2204.04198



In [ ]:
#Datos centrados en cero
x1 = np.linspace(-1, 1,10)
x2 = np.linspace(-1, 1,10) + (2*np.random.random(10)-1)

In [ ]:
print(x1.mean())
x1 = x1 - x1.mean()
print(x1.mean())

In [ ]:
print(x2.mean())
x2 = x2 - x2.mean()
print(x2.mean())

In [ ]:
plt.figure()
plt.plot(x1, x2,"bo")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

In [ ]:
np.stack?

In [ ]:
X = np.stack((x1, x2), axis = 0)
X

In [ ]:
A = np.cov(X)
A

In [ ]:
M = np.matrix(A)
M

In [ ]:
# Autovalores y autovectores
E,U = np.linalg.eig(M)

In [ ]:
U

In [ ]:
U1 = np.array(U[:,0])
U2 = np.array(U[:,1])

In [ ]:
U2

In [ ]:
E

In [ ]:
#comparar a M
#r = (np.matrix(X) @ np.matrix(X).T) /10

In [ ]:
plt.figure()
plt.plot(x1, x2,"bo")
plt.quiver(0,0,U1[0][0],U1[1][0], label = "Vector Proyección 1")
plt.quiver(0,0,U2[0][0],U2[1][0], label = "Vector Proyección 2")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()

In [ ]:
#Proyectamos los datos X en el subespacio reducido de manera que queda una sola variable P
P = U2.T@X
P

In [ ]:
U2[1][0]

In [ ]:
# Pendiente
m = np.arctan(U2[1][0]/U2[0][0])
m

In [ ]:
x2r = x1*np.tan(m)

In [ ]:
plt.figure()
plt.plot(x1, x2,"bo")
plt.quiver(0,0,U1[1][0],U1[0][0], label = "Vector Proyección 1")
plt.quiver(0,0,U2[1][0],U2[0][0], label = "Vector Proyección 2")
plt.plot(x1, x2r,"r-")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()


Observamos que los datos tienen la mayor varianza posible sobre la proyección del primer vector. Esta es una propiedad algebraica que se puede aprovechar para decidir quedarse con los datos proyectados sobre un subespacio mas pequeño que el original con una varianza mayor a ciertdo valor treshold.

El algoritmo quedaría de esta manera (tomado de la referencia indicada arriba):

Algoritmo: Principal component analysis (PCA)

Requiere: Hyperparametro k (la dimensionalidad de los datos proyectados)
En el ejemplo anterior de dos dimensiones que se pueden reducir a una dimension escogeriamos k=1.

Requiere: Matriz X ∈ Rn×m
En el ejemplo anterior de los datos m=2 variables con cada una n=10 records  

Paso 1: En este paso quitamos de manera que cada variable esta centrada en cero.
X ← X − mean(X)

Paso 2: Construimos la matriz de covarianza empírica Σ a partir de XᵀX/n

Paso 3: Calculamos los vectores propios y los ordenamos por el valor de mayor a menor de sus valores propios
V ← Eigenvectors(Σ) 

Paso 4: Construimos la matriz V˜ para hacer la proyección en el espacio reducido de los primeros k-vectores propios
V˜ ← V [:, : k] 

Paso 5: Proyectamos los datos X en el espacio reducido para retornar X˜
X˜ ← X V˜ ∈ Rn×k

## Otras opciones en sklearn


Ver la sección Feature selection en sklearn